# Day 10

## Part 1

"|" is a vertical pipe connecting north and south.

"-" is a horizontal pipe connecting east and west.

"L" is a 90-degree bend connecting north and east.

"J" is a 90-degree bend connecting north and west.

"7" is a 90-degree bend connecting south and west.

"F" is a 90-degree bend connecting south and east.

"." is ground; there is no pipe in this tile.

"S" is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

In [1]:
# lines = [line.rstrip() for line in open('day10_input.txt')]
lines = [line.rstrip() for line in open('day10_sample.txt')]

In [2]:
def find_start():
    grid = []
    
    for i,line in enumerate(lines):
        grid.append(list(line))
        if "S" in line:
            start_loc = (i, line.index("S"))
    return grid, start_loc

In [3]:
def has_connection(grid, row, col, direction):
    pipe = grid[row][col]
    if direction == "north":
        return pipe in ["S", "|", "L", "J"]
    elif direction == "east":
        return pipe in ["S", "-", "L", "F"]
    elif direction == "west":
        return pipe in ["S", "-", "J", "7"]
    elif direction == "south":
        return pipe in ["S", "|", "F", "7"]

In [4]:
def get_neighbors(grid, row, col):
    neighbors = []
    if row > 0 and has_connection(grid, row, col, "north"):
        if has_connection(grid, row - 1, col, "south"):
            neighbors.append((row - 1, col))
    if col > 0 and has_connection(grid, row, col, "west"):
        if has_connection(grid, row, col - 1, "east"):
            neighbors.append((row, col - 1))
    if row < len(grid) - 1 and has_connection(grid, row, col, "south"):
        if has_connection(grid, row + 1, col, "north"):
            neighbors.append((row + 1, col))
    if col < len(grid[0]) - 1 and has_connection(grid, row, col, "east"):
        if has_connection(grid, row, col + 1, "west"):
            neighbors.append((row, col + 1))
    return neighbors

In [5]:
def run_part1():
    grid, start_loc = find_start()
    result = 0
    last_reached_places = [(start_loc, 0)]
    visited = set()
    
    while len(last_reached_places) > 0:
        loc, i = last_reached_places.pop(0)
        row, col = loc
        if (row, col) in visited:
            continue
        visited.add((row, col))
        result = max(result, i)
        for neighbor in get_neighbors(grid, row, col):
            last_reached_places.append((neighbor, i + 1))
    print(result)

In [6]:
run_part1()

8


## Part 2

In [7]:
# lines = [line.rstrip() for line in open('day10_input.txt')]
lines = [line.rstrip() for line in open('day10_sample2.txt')]

In [8]:
def print_grid(grid):
    for i in range(len(grid)):
        row = ""
        for j in range(len(grid[i])):
            row += grid[i][j]
        print(row)

In [9]:
def expand_grid(lines):
    grid = []
    for i, line in enumerate(lines):
        top, mid, bot = "#", "#", "#"
        for ch in line:
            if ch == "S":
                top += "..."
                mid += ".S."
                bot += "..."
            elif ch == ".":
                top += "..."
                mid += "..."
                bot += "..."
            elif ch == "L":
                top += ".|."
                mid += ".L-"
                bot += "..."
            elif ch == "J":
                top += ".|."
                mid += "-J."
                bot += "..."
            elif ch == "F":
                top += "..."
                mid += ".F-"
                bot += ".|."
            elif ch == "7":
                top += "..."
                mid += "-7."
                bot += ".|."
            elif ch == "-":
                top += "..."
                mid += "---"
                bot += "..."
            elif ch == "|":
                top += ".|."
                mid += ".|."
                bot += ".|."
        top += "#"
        mid += "#"
        bot += "#"
        grid.append(list(top))
        grid.append(list(mid))
        grid.append(list(bot))
    
    grid.insert(0, ["#"] * len(grid[0]))
    grid.append(["#"] * len(grid[0]))
    
    return grid

In [10]:
def fill_s(grid):
    for row in range(len(grid)):
        for col in range(len(grid[row])):
            if grid[row][col] == "S":
                start_loc = (row, col)
                d = [(0, 1), (1, 0), (0, -1), (-1, 0)]
                for drow, dcol in d:
                    try:
                        if grid[row + drow * 2][col + dcol * 2] != "." and grid[row + drow * 2][col + dcol * 2] != "#":
                            grid[row + drow][col + dcol] = "S"
                    except:
                        pass
                return grid, start_loc

In [34]:
def find_pipe_in_loop(start_loc, grid):
    queue = [start_loc]
    border = set()
    while len(queue) > 0:
        loc = queue.pop(0)
        row, col = loc
        if (row, col) in border:
            continue
        border.add((row, col))
        d = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        for drow, dcol in d:
            neighbor = (row + drow, col + dcol)
            if neighbor[0] >= 0 and neighbor[0] < len(grid) and neighbor[1] >= 0 and neighbor[1] < len(grid[0]):
                if grid[neighbor[0]][neighbor[1]] != ".":
                    queue.append(neighbor)

    return queue, border

In [49]:
def run_part2():
    lines = [line.rstrip() for line in open('day10_input.txt')]
    # lines = [line.rstrip() for line in open('day10_sample2.txt')]
    
    original_grid = find_start()[0]
    grid = expand_grid(lines)
    grid, start_loc = fill_s(grid)
    print_grid(grid)
    
    queue, border = find_pipe_in_loop(start_loc, grid)

    def border_grid(grid, border):
        new_grid = []
        for i in range(len(grid)):
            row = ""
            for j in range(len(grid[i])):
                if (i, j) in border:
                    row += "X"
                else:
                    row += grid[i][j]
            new_grid.append(row)
        return new_grid

    borders = border_grid(grid, border)
    # print_grid(borders)
    
    def run_bfs(row, col):
        queue = [(row, col)]
        visited = set()
        pipes = set()
        while len(queue) > 0:
            loc = queue.pop(0)
            row, col = loc
            if (row, col) in visited:
                continue
            visited.add((row, col))
            d = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            for drow, dcol in d:
                neighbor = (row + drow, col + dcol)
                if neighbor[0] >= 0 and neighbor[0] < len(grid) and neighbor[1] >= 0 and neighbor[1] < len(grid[0]):
                    if grid[neighbor[0]][neighbor[1]] == "#":
                        return set("border")
                    if neighbor not in border:
                        queue.append(neighbor)
        return pipes
    
    count = 0
    enclosed = []
    orig_rows = len(lines)
    orig_cols = len(lines[0])
    
    for i in range(orig_rows):
        for j in range(orig_cols):
            coords = (i * 3 + 2, j * 3 + 2)
            if coords in border:
                continue
            pipes = run_bfs(coords[0], coords[1])
            if pipes - border == set():
                enclosed.append(coords)
                count += 1
    
    def final_grid(grid):
        new_grid = []
        for i in range(len(grid)):
            row = ""
            for j in range(len(grid[i])):
                if (i, j) in enclosed:
                    row += "I"
                elif (i, j) in border:
                    row += "X"
                else:
                    row += grid[i][j]
            new_grid.append(row)
        return new_grid
    
    new_grid = final_grid(grid)
    # print_grid(new_grid)
    
    print(count)

In [ ]:
run_part2()

######################################################################################################################################################################################################################################################################################################################################################################################################################################
#.......|.......................|..|..|..............|..|...................................|..........................|...........|.....|.......................|........|.................|.....|.....|.................|....................|........|..............|.............................|........|.................|........|..|........|........|.....|.....|........|.................|.............................|.#
#-7.----J.---.......F-.F-....F-.L--J.-J.-7.----7.---.L-.|.-7..F-.F-.F--7..F--7..F-.F-.......|.-7.----7.-7.....F--7.-7..L----------.|.-7..L-----7.-7.---.F-